In [1]:
import os
import numpy as np
import pandas as pd

inputFile = 'senti_post3.csv' 

In [6]:
# Add empty column columns
df = pd.read_csv(inputFile)
df.head(5)

# header_list_new = ['numSentence', 'numWords', 'totSentiment', 'avgSentiment', 'Sfreq0','Sfreq1','Sfreq2','Sfreq3','Sfreq4','Sfreq5']
# for i, name in enumerate(header_list_new):
#     df[name] = 0

,Unnamed: 0,Post_id,user_id,family_code,group_id,message,Post,message_language_code,likes_count,replies_count,...,numSentence,numWords,totSentiment,avgSentiment,Sfreq0,Sfreq1,Sfreq2,Sfreq3,Sfreq4,Sfreq5
0,1,10480,5457,fii-oak-e30849,NaN,Every now and then...,Every now and then...,en,8,3,...,1.0,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
1,3,10491,6503,fii-dtw-f5c055,NaN,Resource for Detroit,Resource for Detroit,en,2,0,...,1.0,2.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
2,4,10532,6048,dsni-bos-b660c6,NaN,Good morning. Hope everyone has a safe n bles...,Good morning. Hope everyone has a safe n bles...,en,6,2,...,1.0,6.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5,10533,5366,fii-dtw-c25633,NaN,Call about this if needed,Call about this if needed,en,1,0,...,1.0,4.0,2.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
4,6,10534,6752,fii-oak-8911e3,NaN,Que tengan un hermoso día mi familia de fii,Have a beautiful day my family of fii,es,6,1,...,1.0,3.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
from pycorenlp import StanfordCoreNLP
nlp = StanfordCoreNLP('http://localhost:9000')

# Function; Output = # sentence, # words, avg.sentimentValue, sentimentHist
def stanford_sentiment(text_str):
    res = nlp.annotate(text_str,
                   properties={
                       'annotators': 'sentiment',
                       'outputFormat': 'json',
                       'timeout': 40000,
                   })
    numSentence = len(res["sentences"])
    numWords = len(text_str.split())
    
    # data arrangement
    arraySentVal = np.zeros(numSentence)

    for i, s in enumerate(res["sentences"]):
        arraySentVal[i] = int(s["sentimentValue"])

    # sum of sentiment values 
    totSentiment = sum(arraySentVal)

    # avg. of sentiment values 
    avgSentiment = np.mean(arraySentVal)

    # frequency of sentimentValue
    bins = [0,1,2,3,4,5,6]
    freq = np.histogram(arraySentVal, bins)[0]    # getting freq. only w/o bins

    return(numSentence, numWords, totSentiment, avgSentiment, freq)

In [8]:
dfLength = len(df)
dfLength

55284

In [35]:

%%time
# dfLength = len(df)
# for i in range(dfLength):
for i in range(54000,55284):
    try:
        numSentence, numWords, totSentiment, avgSentiment, freq = stanford_sentiment(df.clean_text[i].replace('\n'," "))
        df.loc[i,'numSentence'] = numSentence
        df.loc[i,'numWords'] = numWords
        df.loc[i,'totSentiment'] = totSentiment
        df.loc[i,'avgSentiment'] = avgSentiment
        df.loc[i,'Sfreq0'] = freq[0]
        df.loc[i,'Sfreq1'] = freq[1]
        df.loc[i,'Sfreq2'] = freq[2]
        df.loc[i,'Sfreq3'] = freq[3]
        df.loc[i,'Sfreq4'] = freq[4]
        df.loc[i,'Sfreq5'] = freq[5]
    except:
        print("error where i =", i)
        
        

CPU times: user 16.1 s, sys: 1.8 s, total: 17.9 s
Wall time: 3min 4s


In [36]:
outputFile = 'senti_post16.csv'
df.to_csv(outputFile, encoding='utf-8', index=False )